In [2]:
from google.colab import drive
drive.mount('/content/drive')
# then, for convenience
%cd /content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main


In [1]:
!nvidia-smi

Mon Dec  8 18:39:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
%pip install --upgrade pip setuptools wheel

# Let pip choose compatible versions for your Python:
%pip install opacus numpy pandas scikit-learn scipy dython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 30.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 105.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 122.6 MB/s  0:00:00
ERROR: Operation cancelled by user
^C


In [2]:
import os, sys, pathlib, glob
# Ensure we're at the repo root (contains "model/" and "model/ctabgan.py")
assert os.path.exists("model/ctabgan.py"), "Run the unzip cell & cd into the repo root first."

# Make packages explicit for Python
for pkg in ["model", "model/eval", "model/synthesizer"]:
    p = pathlib.Path(pkg); p.mkdir(parents=True, exist_ok=True)
    ip = p / "__init__.py"
    if not ip.exists(): ip.write_text("")

# Put repo on sys.path
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

from model.ctabgan import CTABGAN
print("✅ CTABGAN import OK")


AssertionError: Run the unzip cell & cd into the repo root first.

In [3]:
# This edits model/synthesizer/ctabgan_synthesizer.py in-place.
import re, textwrap, pathlib

path = pathlib.Path("model/synthesizer/ctabgan_synthesizer.py")
src = path.read_text()

# (a) Loss history in __init__
if "self.g_losses" not in src:
    src = re.sub(r"(def\s+__init__\s*\([^\)]*\)\s*:\s*\n)",
                 r"\1        # Track loss history\n        self.g_losses = []\n        self.d_losses = []\n",
                 src, count=1)

# (b) Print/append losses inside epoch loop
if "##__LOSS_HOOK__" not in src:
    src = src.replace("for epoch in range(self.epochs):",
                      "for epoch in range(self.epochs):\n            ##__LOSS_HOOK__")
hook = textwrap.dedent("""
            # Record and print losses each epoch (adjust names if different)
            try:
                g_val = None; d_val = None
                if 'loss_g' in locals(): g_val = float(loss_g.item())
                if 'errG'  in locals() and g_val is None: g_val = float(errG.item())
                if 'g_loss' in locals() and g_val is None: g_val = float(g_loss.item())
                if 'loss_d' in locals(): d_val = float(loss_d.item())
                if 'errD'  in locals() and d_val is None: d_val = float(errD.item())
                if 'd_loss' in locals() and d_val is None: d_val = float(d_loss.item())
                if g_val is not None and d_val is not None:
                    self.g_losses.append(g_val); self.d_losses.append(d_val)
                    print(f"Epoch {epoch+1}/{self.epochs} - G: {g_val:.6f}  D: {d_val:.6f}")
            except Exception:
                pass
""").rstrip("\n")
src = src.replace("##__LOSS_HOOK__", hook)

# (c) Opacus import
if "from opacus import PrivacyEngine" not in src:
    src = src.replace("\nimport", "\nfrom opacus import PrivacyEngine\nimport", 1)

# (d) DP config in __init__
if "self.dp_enabled" not in src:
    src = re.sub(r"(def\s+__init__\s*\([^\)]*\)\s*:\s*\n)",
                 r"\\1        # Differential Privacy config\n"
                 r"        self.dp_enabled = True\n"
                 r"        self.dp_noise = 1.0\n"
                 r"        self.dp_clip = 1.0\n"
                 r"        self.dp_delta = 1e-5\n"
                 r"        self._dp_engine = None\n", src, count=1)

# (e) Wrap discriminator/optimizer/DataLoader with PrivacyEngine.make_private
if "make_private(" not in src:
    # Insert after a likely place where DataLoader is ready.
    src = re.sub(r"(train_loader\s*=\s*.*\n)", r"\\1"
                 r"        # Make discriminator private (DP-SGD)\n"
                 r"        if self.dp_enabled:\n"
                 r"            self._dp_engine = PrivacyEngine()\n"
                 r"            self.D, self.optimizerD, train_loader = self._dp_engine.make_private(\n"
                 r"                module=self.D,\n"
                 r"                optimizer=self.optimizerD,\n"
                 r"                data_loader=train_loader,\n"
                 r"                noise_multiplier=self.dp_noise,\n"
                 r"                max_grad_norm=self.dp_clip,\n"
                 r"                poisson_sampling=True,\n"
                 r"            )\n", src, count=1)

# (f) ε report each epoch
if "get_epsilon(" not in src:
    src = re.sub(r"(for\s+epoch\s+in\s+range\(\s*self\.epochs\s*\)\s*:\s*\n)",
                 r"\\1            if getattr(self, '_dp_engine', None) is not None:\n"
                 r"                try:\n"
                 r"                    _eps = self._dp_engine.get_epsilon(delta=self.dp_delta)\n"
                 r"                    print(f\"[DP] epoch {epoch+1}: ε={_eps:.2f}, δ={self.dp_delta}, σ={self.dp_noise}, C={self.dp_clip}\")\n"
                 r"                except Exception:\n"
                 r"                    pass\n", src, count=1)

path.write_text(src)
print("✅ Patched:", path)


FileNotFoundError: [Errno 2] No such file or directory: 'model/synthesizer/ctabgan_synthesizer.py'

In [ ]:
import pandas as pd, os
from model.ctabgan import CTABGAN

def infer_config(df: pd.DataFrame, label_hint=None):
    # Guess label
    label = None
    for cand in [label_hint, "Diabetes_binary", "Diabetes_012", "label", "target"]:
        if cand and cand in df.columns:
            label = cand; break

    categorical_columns = df.select_dtypes(include=['object','category','bool']).columns.tolist()
    integer_columns = [c for c in df.columns if pd.api.types.is_integer_dtype(df[c])]
    log_columns = []           # add strictly-positive skewed columns if any
    mixed_columns = {}         # e.g., {'amount':[0.0]} for zero-inflated positives
    general_columns = []       # continuous that are not log/mixed
    non_categorical_columns = [c for c in df.columns if c not in categorical_columns]
    problem_type = {"Classification": label} if label else None

    return dict(
        categorical_columns=categorical_columns,
        integer_columns=integer_columns,
        log_columns=log_columns,
        mixed_columns=mixed_columns,
        general_columns=general_columns,
        non_categorical_columns=non_categorical_columns,
        problem_type=problem_type
    )

def train_and_sample(csv_path, dp_noise=1.0, dp_clip=1.0, epochs=50, out_name=None):
    df = pd.read_csv(csv_path)
    cfg = infer_config(df)
    print("Label:", cfg["problem_type"])

    g = CTABGAN(
        raw_csv_path=csv_path,
        test_ratio=0.2,
        **{k: cfg[k] for k in ["categorical_columns","log_columns","mixed_columns",
                               "general_columns","non_categorical_columns","integer_columns"]},
        problem_type=cfg["problem_type"]
    )

    # tweak DP settings on synthesizer
    g.synthesizer.dp_enabled = True
    g.synthesizer.dp_noise   = float(dp_noise)
    g.synthesizer.dp_clip    = float(dp_clip)
    g.synthesizer.dp_delta   = 1e-5
    g.synthesizer.epochs     = int(epochs)

    g.fit()
    syn = g.generate_samples()          # same #rows as real by default
    os.makedirs("Fake_Datasets", exist_ok=True)
    out = f"Fake_Datasets/{out_name or os.path.basename(csv_path).replace('.csv','_synthetic.csv')}"
    syn.to_csv(out, index=False)
    print("Saved synthetic:", out)
    return out, g


In [ ]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ks_2samp

def tvd(p, q):
    keys = sorted(set(p.index).union(q.index))
    p = p.reindex(keys, fill_value=0).values
    q = q.reindex(keys, fill_value=0).values
    return 0.5 * np.abs(p - q).sum()

def range_coverage(real, syn):
    rmin, rmax = np.nanmin(real), np.nanmax(real)
    smin, smax = np.nanmin(syn),  np.nanmax(syn)
    if not np.isfinite([rmin,rmax,smin,smax]).all() or rmax<=rmin:
        return 0.0
    span = rmax - rmin
    left  = max(0.0, (smin - rmin) / span)
    right = max(0.0, (rmax - smax) / span)
    return float(np.clip(1.0 - 0.5*(left+right), 0, 1))

def anomaly_similarity(real, syn):
    qr = np.nanpercentile(real, [25,50,75])
    qs = np.nanpercentile(syn,  [25,50,75])
    def rate(x, q):
        iqr = q[2]-q[0]
        if iqr<=0: return 0.0
        low, high = q[0]-1.5*iqr, q[2]+1.5*iqr
        return float(((x<low)|(x>high)).mean())
    rr, rs = rate(real, qr), rate(syn, qs)
    return float(1.0 - np.clip(abs(rr-rs), 0, 1))

def corr_similarity(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if len(num)<2: return 0.5
    Cr = df_r[num].corr().fillna(0).to_numpy()
    Cs = df_s[num].corr().fillna(0).to_numpy()
    diff = np.linalg.norm(Cr - Cs, ord="fro")
    base = np.linalg.norm(np.abs(Cr)+np.abs(Cs), ord="fro") + 1e-8
    return float(np.clip(1.0 - diff/base, 0, 1))

def category_coverage(df_r, df_s, cols):
    if not cols: return 1.0
    covs = []
    for c in cols:
        r = set(df_r[c].astype(str).unique())
        s = set(df_s[c].astype(str).unique())
        covs.append(len(r & s)/max(1,len(r)))
    return float(np.mean(covs))

def quantile_diff_score(real, syn, q=100):
    qs = np.linspace(0,100,q+1)
    r = np.nanpercentile(real, qs); s = np.nanpercentile(syn, qs)
    d = np.mean(np.abs(r - s))
    scale = np.nanpercentile(real, 75) - np.nanpercentile(real, 25)
    if not np.isfinite(scale) or scale<=0: scale = np.nanstd(real) + 1e-8
    return float(np.clip(1.0 - d/(scale+1e-8), 0, 1))

def dcr_score(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.5
    R = df_r[num].dropna().to_numpy()
    S = df_s[num].dropna().to_numpy()
    if len(R)==0 or len(S)==0: return 0.5
    sc = StandardScaler().fit(np.vstack([R,S]))
    Rz, Sz = sc.transform(R), sc.transform(S)
    nn = NearestNeighbors(n_neighbors=1).fit(Rz)
    d_sr = nn.kneighbors(Sz, return_distance=True)[0].ravel()
    d_rr = nn.kneighbors(Rz, return_distance=True)[0].ravel()
    shift = np.median(d_sr) - np.median(d_rr)
    return float(1.0/(1.0+np.exp(-4.0*(shift-0.05))))  # larger shift ⇒ more private

def duplicate_overlap(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.0
    R = df_r[num].fillna(0).to_numpy()
    S = df_s[num].fillna(0).to_numpy()
    R = (R - R.min(0))/(R.ptp(0)+1e-8); S = (S - S.min(0))/(S.ptp(0)+1e-8)
    R = (R*64).astype(int); S = (S*64).astype(int)
    rkeys = {tuple(row) for row in R}
    skeys = {tuple(row) for row in S}
    ov = len(rkeys & skeys)/max(1,len(skeys))  # lower is better
    return float(ov)

def rate_QP(real_csv, syn_csv, dp_sigma_release=0.0):
    real = pd.read_csv(real_csv); syn = pd.read_csv(syn_csv)

    cat_cols = real.select_dtypes(include=['object','category','bool']).columns.tolist()
    num_cols = [c for c in real.columns if c not in cat_cols and pd.api.types.is_numeric_dtype(real[c])]

    # Quality
    ks_vals = [ks_2samp(real[c].dropna().astype(float), syn[c].dropna().astype(float)).statistic
               for c in num_cols if len(real[c].dropna())>30 and len(syn[c].dropna())>30]
    Q_num = 1.0 - (np.mean(ks_vals) if ks_vals else 1.0)

    tvd_vals = [tvd(real[c].value_counts(normalize=True), syn[c].value_counts(normalize=True)) for c in cat_cols]
    Q_cat = 1.0 - (np.mean(tvd_vals) if tvd_vals else 0.0)

    Q_range = np.mean([range_coverage(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
    Q_anom  = np.mean([anomaly_similarity(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
    Q_corr  = corr_similarity(real, syn)
    Q_cov   = category_coverage(real, syn, cat_cols)

    w = dict(num=0.20, cat=0.20, range=0.15, anom=0.15, corr=0.20, cov=0.10)
    Q = (w['num']*Q_num + w['cat']*Q_cat + w['range']*Q_range +
         w['anom']*Q_anom + w['corr']*Q_corr + w['cov']*Q_cov)

    # Privacy
    P_dcr = dcr_score(real, syn)
    qd_vals = [quantile_diff_score(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]
    P_qd = np.mean(qd_vals) if qd_vals else 0.5
    P_dup = 1.0 - np.clip(duplicate_overlap(real, syn), 0, 1)
    wp = dict(dcr=0.5, qd=0.3, dup=0.2)
    P = wp['dcr']*P_dcr + wp['qd']*P_qd + wp['dup']*P_dup

    return dict(Q_overall=Q, Q_num=Q_num, Q_cat=Q_cat, Q_range=Q_range, Q_anom=Q_anom, Q_corr=Q_corr, Q_div=Q_cov,
                P_overall=P, P_DCR=P_dcr, P_Qdelta=P_qd, P_noDup=P_dup)


In [ ]:
import pandas as pd
import re

def auto_label(df: pd.DataFrame, label_hint=None):
    """
    Return the name of the label column for CTAB-GAN+ classification.
    Tries explicit hint, common diabetes names, and a heuristic (name contains 'diabetes' and low cardinality).
    Raises with a helpful message if nothing fits.
    """
    # normalize column names (strip spaces)
    df = df.copy()
    df.columns = df.columns.str.strip()

    # 1) explicit hint
    if label_hint and label_hint in df.columns:
        return label_hint

    # 2) common diabetes labels
    common = ["Diabetes_binary", "diabetes_binary", "Diabetes_012", "diabetes_012", "Diabetes"]
    for c in df.columns:
        if c in common:
            # must be classification-like cardinality
            if df[c].nunique(dropna=True) in (2, 3):
                return c

    # 3) heuristic: column name contains 'diabetes' and is 2 or 3 classes
    for c in df.columns:
        if "diab" in c.lower() and df[c].nunique(dropna=True) in (2, 3):
            return c

    # 4) last-resort: smallest-cardinality non-constant column (but still classification-like)
    candidates = [(c, df[c].nunique(dropna=True)) for c in df.columns]
    candidates = [(c, k) for c, k in candidates if 1 < k <= 10]  # avoid continuous
    if candidates:
        candidates.sort(key=lambda x: x[1])
        # prefer exactly 2 or 3 classes if present
        for c,k in candidates:
            if k in (2,3):
                return c
        # otherwise take the smallest discrete
        return candidates[0][0]

    raise ValueError(
        "Could not infer a classification label column. "
        "Pass label_hint='Diabetes_binary' or 'Diabetes_012' to train_and_sample(...)."
    )

def infer_config(df: pd.DataFrame, label_hint=None):
    df = df.copy()
    df.columns = df.columns.str.strip()

    # Pick the label (binary or 3-class)
    label = auto_label(df, label_hint=label_hint)

    # Ensure label is integer-coded (0/1 or 0/1/2)
    if not pd.api.types.is_integer_dtype(df[label]):
        # try safe cast for typical 0.0/1.0 or strings like "0"/"1"
        try:
            df[label] = pd.to_numeric(df[label], errors="raise").round().astype(int)
        except Exception:
            # fallback: category codes
            df[label] = df[label].astype("category").cat.codes

    # Build column type lists
    categorical_columns = df.select_dtypes(include=['object','category','bool']).columns.tolist()
    integer_columns = [c for c in df.columns if pd.api.types.is_integer_dtype(df[c])]
    log_columns = []
    mixed_columns = {}
    general_columns = []
    non_categorical_columns = [c for c in df.columns if c not in categorical_columns]

    return dict(
        problem_type={"Classification": label},
        categorical_columns=categorical_columns,
        integer_columns=integer_columns,
        log_columns=log_columns,
        mixed_columns=mixed_columns,
        general_columns=general_columns,
        non_categorical_columns=non_categorical_columns
    )

def train_and_sample(csv_path, CTABGAN_cls, dp_noise=1.0, dp_clip=1.0, epochs=60, out_name=None, label_hint=None):
    df = pd.read_csv(csv_path)
    cfg = infer_config(df, label_hint=label_hint)
    print("Label detected:", cfg["problem_type"])

    g = CTABGAN_cls(
        raw_csv_path=csv_path,
        test_ratio=0.2,
        categorical_columns=cfg["categorical_columns"],
        integer_columns=cfg["integer_columns"],
        log_columns=cfg["log_columns"],
        mixed_columns=cfg["mixed_columns"],
        general_columns=cfg["general_columns"],
        non_categorical_columns=cfg["non_categorical_columns"],
        problem_type=cfg["problem_type"],   # 👈 always non-None now
    )

    # DP knobs on the synthesizer (assumes your DP patch is in place)
    s = g.synthesizer
    s.dp_enabled = True
    s.dp_noise   = float(dp_noise)
    s.dp_clip    = float(dp_clip)
    s.dp_delta   = 1e-5
    s.epochs     = int(epochs)

    g.fit()
    syn = g.generate_samples()
    out_path = out_name or (csv_path.rsplit(".",1)[0] + "_CTABGAN_DP.csv")
    pd.DataFrame(syn).to_csv(out_path, index=False)
    print("Saved synthetic:", out_path)
    return out_path, g


In [ ]:
from model.ctabgan import CTABGAN

real_csvs = [
    "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin.csv",
]

# Provide hints so we pick the intended label quickly
hints = {
    "diabetes_binary_health_indicators_BRFSS2015.csv": "Diabetes_binary",
    "diabetes_binary_5050split_health_indicators_BRFSS2015.csv": "Diabetes_binary",
    "diabetes_012_health_indicators_BRFSS2015.csv": "Diabetes_012",
}

DP_NOISE, DP_CLIP, EPOCHS = 1.0, 1.0, 80

for rp in real_csvs:
    hint = None
    for k,v in hints.items():
        if k in rp:
            hint = v
            break
    out_syn, model = train_and_sample(
        csv_path=rp,
        CTABGAN_cls=CTABGAN,
        dp_noise=DP_NOISE,
        dp_clip=DP_CLIP,
        epochs=EPOCHS,
        out_name=rp.replace(".csv", "_CTABGAN_DP.csv"),
        label_hint=hint,   # 👈 this ensures `problem_type` is set
    )


In [ ]:
# Q (quality) & P (privacy) ratings for Breast Cancer Wisconsin (real vs CTAB-GAN+ DP synthetic)

import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ks_2samp

# --- set your paths (update if your files live elsewhere) ---
real_path = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin.csv"
syn_path  = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin_CTABGAN_DP.csv"

real = pd.read_csv(real_path)
syn  = pd.read_csv(syn_path)

# ---------- helpers ----------
def tvd(p, q):
    keys = sorted(set(p.index).union(q.index))
    p = p.reindex(keys, fill_value=0).values
    q = q.reindex(keys, fill_value=0).values
    return 0.5 * np.abs(p - q).sum()

def range_coverage(real, syn):
    rmin, rmax = np.nanmin(real), np.nanmax(real)
    smin, smax = np.nanmin(syn),  np.nanmax(syn)
    if not np.isfinite([rmin,rmax,smin,smax]).all() or rmax<=rmin:
        return 0.0
    span = rmax - rmin
    left  = max(0.0, (smin - rmin) / span)
    right = max(0.0, (rmax - smax) / span)
    return float(np.clip(1.0 - 0.5*(left+right), 0, 1))

def anomaly_similarity(real, syn):
    qr = np.nanpercentile(real, [25,50,75])
    qs = np.nanpercentile(syn,  [25,50,75])
    def rate(x, q):
        iqr = q[2]-q[0]
        if iqr<=0: return 0.0
        low, high = q[0]-1.5*iqr, q[2]+1.5*iqr
        return float(((x<low)|(x>high)).mean())
    rr, rs = rate(real, qr), rate(syn, qs)
    return float(1.0 - np.clip(abs(rr-rs), 0, 1))

def corr_similarity(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if len(num)<2: return 0.5
    Cr = df_r[num].corr().fillna(0).to_numpy()
    Cs = df_s[num].corr().fillna(0).to_numpy()
    diff = np.linalg.norm(Cr - Cs, ord="fro")
    base = np.linalg.norm(np.abs(Cr)+np.abs(Cs), ord="fro") + 1e-8
    return float(np.clip(1.0 - diff/base, 0, 1))

def category_coverage(df_r, df_s, cols):
    if not cols: return 1.0
    covs = []
    for c in cols:
        r = set(df_r[c].astype(str).unique())
        s = set(df_s[c].astype(str).unique())
        covs.append(len(r & s)/max(1,len(r)))
    return float(np.mean(covs))

def quantile_diff_score(real, syn, q=100):
    qs = np.linspace(0,100,q+1)
    r = np.nanpercentile(real, qs); s = np.nanpercentile(syn, qs)
    d = np.mean(np.abs(r - s))
    scale = np.nanpercentile(real, 75) - np.nanpercentile(real, 25)
    if not np.isfinite(scale) or scale<=0: scale = np.nanstd(real) + 1e-8
    return float(np.clip(1.0 - d/(scale+1e-8), 0, 1))

def dcr_score(df_r, df_s):
    # Distance to closest real record shift (synthetic→real vs real→real); larger shift = more private
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.5
    R = df_r[num].dropna().to_numpy()
    S = df_s[num].dropna().to_numpy()
    if len(R)==0 or len(S)==0: return 0.5
    sc = StandardScaler().fit(np.vstack([R,S]))
    Rz, Sz = sc.transform(R), sc.transform(S)
    nn = NearestNeighbors(n_neighbors=1).fit(Rz)
    d_sr = nn.kneighbors(Sz, return_distance=True)[0].ravel()
    d_rr = nn.kneighbors(Rz, return_distance=True)[0].ravel()
    shift = np.median(d_sr) - np.median(d_rr)
    return float(1.0/(1.0+np.exp(-4.0*(shift-0.05))))

def duplicate_overlap(df_r, df_s):
    # Rough near-duplicate proxy on numeric features via coarse binning
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.0
    R = df_r[num].fillna(0).to_numpy()
    S = df_s[num].fillna(0).to_numpy()
    R = (R - R.min(0))/(R.ptp(0)+1e-8); S = (S - S.min(0))/(S.ptp(0)+1e-8)
    R = (R*64).astype(int); S = (S*64).astype(int)
    rkeys = {tuple(row) for row in R}
    skeys = {tuple(row) for row in S}
    ov = len(rkeys & skeys)/max(1,len(skeys))  # lower is better
    return float(ov)

# ---------- compute Q ----------
cat_cols = real.select_dtypes(include=['object','category','bool']).columns.tolist()
num_cols = [c for c in real.columns if c not in cat_cols and pd.api.types.is_numeric_dtype(real[c])]

ks_vals = []
for c in num_cols:
    r = real[c].dropna().astype(float); s = syn[c].dropna().astype(float)
    if len(r)>30 and len(s)>30:
        ks_vals.append(ks_2samp(r, s).statistic)
Q_num = 1.0 - (np.mean(ks_vals) if ks_vals else 1.0)

tvd_vals = []
for c in cat_cols:
    tvd_vals.append(tvd(real[c].value_counts(normalize=True), syn[c].value_counts(normalize=True)))
Q_cat = 1.0 - (np.mean(tvd_vals) if tvd_vals else 0.0)

Q_range = np.mean([range_coverage(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_anom  = np.mean([anomaly_similarity(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_corr  = corr_similarity(real, syn)
Q_div   = category_coverage(real, syn, cat_cols)

w = dict(num=0.20, cat=0.20, range=0.15, anom=0.15, corr=0.20, cov=0.10)
Q_overall = (w['num']*Q_num + w['cat']*Q_cat + w['range']*Q_range +
             w['anom']*Q_anom + w['corr']*Q_corr + w['cov']*Q_div)

# ---------- compute P ----------
P_DCR    = dcr_score(real, syn)
P_Qdelta = np.mean([quantile_diff_score(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 0.5
P_noDup  = 1.0 - np.clip(duplicate_overlap(real, syn), 0, 1)
wp = dict(dcr=0.5, qd=0.3, dup=0.2)
P_overall = wp['dcr']*P_DCR + wp['qd']*P_Qdelta + wp['dup']*P_noDup

# ---------- report ----------
report = pd.DataFrame([{
    "dataset": Path(real_path).name,
    "synthetic": Path(syn_path).name,
    "Q_overall": round(Q_overall,3),
    "Q_num(1-KS)": round(Q_num,3),
    "Q_cat(1-TVD)": round(Q_cat,3),
    "Q_range": round(Q_range,3),
    "Q_anomaly": round(Q_anom,3),
    "Q_corr": round(Q_corr,3),
    "Q_diversity": round(Q_div,3),
    "P_overall": round(P_overall,3),
    "P_DCR": round(P_DCR,3),
    "P_Qdelta": round(P_Qdelta,3),
    "P_noDup": round(P_noDup,3),
    "n_real": len(real),
    "n_syn": len(syn),
    "num_cols_used": len(num_cols),
    "cat_cols_used": len(cat_cols)
}])
print(report.to_string(index=False))

# Save for download
out_csv = "/mnt/data/breast_cancer_CTABGAN_DP_QP_report.csv"
report.to_csv(out_csv, index=False)
print("\nSaved report to:", out_csv)


In [ ]:
# Q (quality) & P (privacy) ratings for Real vs CTAB-GAN+ (DP-SGD) Synthetic
# --- Set your paths here ---
real_path = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin.csv"
syn_path  = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin_CTABGAN_DP.csv"
out_csv   = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast_cancer_CTABGAN_DP_QP_report.csv"

import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ks_2samp

# ---------- helpers ----------
def tvd(p, q):
    keys = sorted(set(p.index).union(q.index))
    p = p.reindex(keys, fill_value=0).values
    q = q.reindex(keys, fill_value=0).values
    return 0.5 * np.abs(p - q).sum()

def range_coverage(real, syn):
    rmin, rmax = np.nanmin(real), np.nanmax(real)
    smin, smax = np.nanmin(syn),  np.nanmax(syn)
    if not np.isfinite([rmin,rmax,smin,smax]).all() or rmax<=rmin:
        return 0.0
    span = rmax - rmin
    left  = max(0.0, (smin - rmin) / span)
    right = max(0.0, (rmax - smax) / span)
    return float(np.clip(1.0 - 0.5*(left+right), 0, 1))

def anomaly_similarity(real, syn):
    qr = np.nanpercentile(real, [25,50,75])
    qs = np.nanpercentile(syn,  [25,50,75])
    def rate(x, q):
        iqr = q[2]-q[0]
        if iqr<=0: return 0.0
        low, high = q[0]-1.5*iqr, q[2]+1.5*iqr
        return float(((x<low)|(x>high)).mean())
    rr, rs = rate(real, qr), rate(syn, qs)
    return float(1.0 - np.clip(abs(rr-rs), 0, 1))

def corr_similarity(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if len(num)<2: return 0.5
    Cr = df_r[num].corr().fillna(0).to_numpy()
    Cs = df_s[num].corr().fillna(0).to_numpy()
    diff = np.linalg.norm(Cr - Cs, ord="fro")
    base = np.linalg.norm(np.abs(Cr)+np.abs(Cs), ord="fro") + 1e-8
    return float(np.clip(1.0 - diff/base, 0, 1))

def category_coverage(df_r, df_s, cols):
    if not cols: return 1.0
    covs = []
    for c in cols:
        r = set(df_r[c].astype(str).unique())
        s = set(df_s[c].astype(str).unique())
        covs.append(len(r & s)/max(1,len(r)))
    return float(np.mean(covs))

def quantile_diff_score(real, syn, q=100):
    qs = np.linspace(0,100,q+1)
    r = np.nanpercentile(real, qs); s = np.nanpercentile(syn, qs)
    d = np.mean(np.abs(r - s))
    scale = np.nanpercentile(real, 75) - np.nanpercentile(real, 25)
    if not np.isfinite(scale) or scale<=0: scale = np.nanstd(real) + 1e-8
    return float(np.clip(1.0 - d/(scale+1e-8), 0, 1))

def dcr_score(df_r, df_s):
    # Distance to closest real record shift (synthetic→real vs real→real); larger shift = more private
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.5
    R = df_r[num].dropna().to_numpy()
    S = df_s[num].dropna().to_numpy()
    if len(R)==0 or len(S)==0: return 0.5
    sc = StandardScaler().fit(np.vstack([R,S]))
    Rz, Sz = sc.transform(R), sc.transform(S)
    nn = NearestNeighbors(n_neighbors=1).fit(Rz)
    d_sr = nn.kneighbors(Sz, return_distance=True)[0].ravel()
    d_rr = nn.kneighbors(Rz, return_distance=True)[0].ravel()
    shift = np.median(d_sr) - np.median(d_rr)
    return float(1.0/(1.0+np.exp(-4.0*(shift-0.05))))

def duplicate_overlap(df_r, df_s):
    """
    Rough near-duplicate proxy on numeric features via coarse binning.
    **NumPy 2.x safe**: uses np.ptp(..., axis=0) instead of ndarray.ptp(...)
    """
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.0
    R = df_r[num].fillna(0).to_numpy()
    S = df_s[num].fillna(0).to_numpy()
    # --- minimal change per your error: use np.ptp instead of R.ptp / S.ptp ---
    R = (R - R.min(0)) / (np.ptp(R, axis=0) + 1e-8)
    S = (S - S.min(0)) / (np.ptp(S, axis=0) + 1e-8)
    R = (R*64).astype(int); S = (S*64).astype(int)
    rkeys = {tuple(row) for row in R}
    skeys = {tuple(row) for row in S}
    ov = len(rkeys & skeys)/max(1,len(skeys))  # lower is better
    return float(ov)

# ---------- load data ----------
real = pd.read_csv(real_path)
syn  = pd.read_csv(syn_path)

# ---------- compute Q ----------
cat_cols = real.select_dtypes(include=['object','category','bool']).columns.tolist()
num_cols = [c for c in real.columns if c not in cat_cols and pd.api.types.is_numeric_dtype(real[c])]

ks_vals = []
for c in num_cols:
    r = real[c].dropna().astype(float); s = syn[c].dropna().astype(float)
    if len(r)>30 and len(s)>30:
        ks_vals.append(ks_2samp(r, s).statistic)
Q_num = 1.0 - (np.mean(ks_vals) if ks_vals else 1.0)

tvd_vals = []
for c in cat_cols:
    tvd_vals.append(tvd(real[c].value_counts(normalize=True), syn[c].value_counts(normalize=True)))
Q_cat = 1.0 - (np.mean(tvd_vals) if tvd_vals else 0.0)

Q_range = np.mean([range_coverage(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_anom  = np.mean([anomaly_similarity(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_corr  = corr_similarity(real, syn)
Q_div   = category_coverage(real, syn, cat_cols)

w = dict(num=0.20, cat=0.20, range=0.15, anom=0.15, corr=0.20, cov=0.10)
Q_overall = (w['num']*Q_num + w['cat']*Q_cat + w['range']*Q_range +
             w['anom']*Q_anom + w['corr']*Q_corr + w['cov']*Q_div)

# ---------- compute P ----------
P_DCR    = dcr_score(real, syn)
P_Qdelta = np.mean([quantile_diff_score(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 0.5
P_noDup  = 1.0 - np.clip(duplicate_overlap(real, syn), 0, 1)
wp = dict(dcr=0.5, qd=0.3, dup=0.2)
P_overall = wp['dcr']*P_DCR + wp['qd']*P_Qdelta + wp['dup']*P_noDup

# ---------- report ----------
report = pd.DataFrame([{
    "dataset": Path(real_path).name,
    "synthetic": Path(syn_path).name,
    "Q_overall": round(Q_overall,3),
    "Q_num(1-KS)": round(Q_num,3),
    "Q_cat(1-TVD)": round(Q_cat,3),
    "Q_range": round(Q_range,3),
    "Q_anomaly": round(Q_anom,3),
    "Q_corr": round(Q_corr,3),
    "Q_diversity": round(Q_div,3),
    "P_overall": round(P_overall,3),
    "P_DCR": round(P_DCR,3),
    "P_Qdelta": round(P_Qdelta,3),
    "P_noDup": round(P_noDup,3),
    "n_real": len(real),
    "n_syn": len(syn),
    "num_cols_used": len(num_cols),
    "cat_cols_used": len(cat_cols)
}])

print(report.to_string(index=False))
report.to_csv(out_csv, index=False)
print("\nSaved report to:", out_csv)


In [4]:
# Plot CTAB-GAN+ (DP-SGD) loss curves for Breast Cancer dataset
# Tries (1) in-memory losses on the trained model; else (2) CSV logs on disk.

import os, sys, glob, json, pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def _has_attr_chain(obj, chain):
    cur = obj
    for name in chain.split('.'):
        if not hasattr(cur, name):
            return False
        cur = getattr(cur, name)
    return True

def _get_attr_chain(obj, chain):
    cur = obj
    for name in chain.split('.'):
        cur = getattr(cur, name)
    return cur

def load_losses_from_memory():
    # Accept either `ctab` or `model` variable names
    for varname in ("ctab", "model"):
        if varname in globals():
            obj = globals()[varname]
            if _has_attr_chain(obj, "synthesizer.g_losses") and _has_attr_chain(obj, "synthesizer.d_losses"):
                g = _get_attr_chain(obj, "synthesizer.g_losses")
                d = _get_attr_chain(obj, "synthesizer.d_losses")
                if isinstance(g, (list, tuple)) and isinstance(d, (list, tuple)) and len(g) and len(d):
                    return list(range(1, 1+min(len(g), len(d)))), np.array(g[:len(d)]), np.array(d[:len(g)])
    return None

def load_losses_from_csv():
    candidates = []
    # Common places/names; add your path if you saved elsewhere
    candidates += glob.glob("outputs/*loss*.csv")
    candidates += glob.glob("Fake_Datasets/*loss*.csv")
    candidates += glob.glob("**/*loss*.csv", recursive=True)
    candidates = [p for p in candidates if os.path.isfile(p)]
    for p in sorted(candidates):
        try:
            df = pd.read_csv(p)
            cols = [c.lower() for c in df.columns]
            # Expect epoch, g_loss, d_loss (case-insensitive)
            def col(name):
                for c in df.columns:
                    if c.lower()==name: return c
                return None
            ec = col("epoch"); gc = col("g_loss"); dc = col("d_loss")
            if ec and gc and dc:
                e = df[ec].to_numpy()
                g = df[gc].to_numpy()
                d = df[dc].to_numpy()
                k = int(min(len(e), len(g), len(d)))
                if k>0:
                    return e[:k], g[:k], d[:k], p
        except Exception:
            continue
    return None

loaded = load_losses_from_memory()
source = "memory"
csv_path = None
if loaded is None:
    csv_loaded = load_losses_from_csv()
    if csv_loaded is not None:
        e, g, d, csv_path = csv_loaded
        source = f"csv:{csv_path}"
    else:
        e = g = d = None

if e is None:
    msg = (
        "No loss history found.\n\n"
        "I looked for:\n"
        "  1) an in-memory trained object named `ctab` or `model` with `synthesizer.g_losses` and `synthesizer.d_losses`\n"
        "  2) a CSV like `*loss*.csv` containing columns: epoch,g_loss,d_loss\n\n"
        "If you patched the repo to record losses, make sure to:\n"
        "  - keep your trained object alive (e.g., variable `ctab`), or\n"
        "  - write the losses to CSV during training and point me to that file.\n"
    )
    raise RuntimeError(msg)

# If we got memory-based losses, unpack them now
if loaded is not None:
    e, g, d = loaded

# Plot one chart with both losses
plt.figure()
plt.plot(e, g, label="G (generator)")
plt.plot(e, d, label="D (discriminator)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("CTAB-GAN+ (DP-SGD) — Training Loss (Breast Cancer)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Loss source: {source}")
if csv_path:
    print(f"Used log file: {csv_path}")


RuntimeError: No loss history found.

I looked for:
  1) an in-memory trained object named `ctab` or `model` with `synthesizer.g_losses` and `synthesizer.d_losses`
  2) a CSV like `*loss*.csv` containing columns: epoch,g_loss,d_loss

If you patched the repo to record losses, make sure to:
  - keep your trained object alive (e.g., variable `ctab`), or
  - write the losses to CSV during training and point me to that file.


In [5]:
# Q (quality) & P (privacy) ratings for Real vs CTAB-GAN+ (DP-SGD) Synthetic
# --- Set your paths here ---
real_path = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin.csv"
syn_path  = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast-cancer-wisconsin_CTABGAN_DP.csv"
out_csv   = "/content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast_cancer_CTABGAN_DP_QP_report.csv"

import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ks_2samp

# ---------- helpers ----------
def tvd(p, q):
    keys = sorted(set(p.index).union(q.index))
    p = p.reindex(keys, fill_value=0).values
    q = q.reindex(keys, fill_value=0).values
    return 0.5 * np.abs(p - q).sum()

def range_coverage(real, syn):
    rmin, rmax = np.nanmin(real), np.nanmax(real)
    smin, smax = np.nanmin(syn),  np.nanmax(syn)
    if not np.isfinite([rmin,rmax,smin,smax]).all() or rmax<=rmin:
        return 0.0
    span = rmax - rmin
    left  = max(0.0, (smin - rmin) / span)
    right = max(0.0, (rmax - smax) / span)
    return float(np.clip(1.0 - 0.5*(left+right), 0, 1))

def anomaly_similarity(real, syn):
    qr = np.nanpercentile(real, [25,50,75])
    qs = np.nanpercentile(syn,  [25,50,75])
    def rate(x, q):
        iqr = q[2]-q[0]
        if iqr<=0: return 0.0
        low, high = q[0]-1.5*iqr, q[2]+1.5*iqr
        return float(((x<low)|(x>high)).mean())
    rr, rs = rate(real, qr), rate(syn, qs)
    return float(1.0 - np.clip(abs(rr-rs), 0, 1))

def corr_similarity(df_r, df_s):
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if len(num)<2: return 0.5
    Cr = df_r[num].corr().fillna(0).to_numpy()
    Cs = df_s[num].corr().fillna(0).to_numpy()
    diff = np.linalg.norm(Cr - Cs, ord="fro")
    base = np.linalg.norm(np.abs(Cr)+np.abs(Cs), ord="fro") + 1e-8
    return float(np.clip(1.0 - diff/base, 0, 1))

def category_coverage(df_r, df_s, cols):
    if not cols: return 1.0
    covs = []
    for c in cols:
        r = set(df_r[c].astype(str).unique())
        s = set(df_s[c].astype(str).unique())
        covs.append(len(r & s)/max(1,len(r)))
    return float(np.mean(covs))

def quantile_diff_score(real, syn, q=100):
    qs = np.linspace(0,100,q+1)
    r = np.nanpercentile(real, qs); s = np.nanpercentile(syn, qs)
    d = np.mean(np.abs(r - s))
    scale = np.nanpercentile(real, 75) - np.nanpercentile(real, 25)
    if not np.isfinite(scale) or scale<=0: scale = np.nanstd(real) + 1e-8
    return float(np.clip(1.0 - d/(scale+1e-8), 0, 1))

def dcr_score(df_r, df_s):
    # Distance to closest real record shift (synthetic→real vs real→real); larger shift = more private
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.5
    R = df_r[num].dropna().to_numpy()
    S = df_s[num].dropna().to_numpy()
    if len(R)==0 or len(S)==0: return 0.5
    sc = StandardScaler().fit(np.vstack([R,S]))
    Rz, Sz = sc.transform(R), sc.transform(S)
    nn = NearestNeighbors(n_neighbors=1).fit(Rz)
    d_sr = nn.kneighbors(Sz, return_distance=True)[0].ravel()
    d_rr = nn.kneighbors(Rz, return_distance=True)[0].ravel()
    shift = np.median(d_sr) - np.median(d_rr)
    return float(1.0/(1.0+np.exp(-4.0*(shift-0.05))))

def duplicate_overlap(df_r, df_s):
    """
    Rough near-duplicate proxy on numeric features via coarse binning.
    **NumPy 2.x safe**: uses np.ptp(..., axis=0) instead of ndarray.ptp(...).
    """
    num = df_r.select_dtypes(include=[np.number]).columns.tolist()
    if not num: return 0.0
    R = df_r[num].fillna(0).to_numpy()
    S = df_s[num].fillna(0).to_numpy()
    R = (R - R.min(0)) / (np.ptp(R, axis=0) + 1e-8)
    S = (S - S.min(0)) / (np.ptp(S, axis=0) + 1e-8)
    R = (R*64).astype(int); S = (S*64).astype(int)
    rkeys = {tuple(row) for row in R}
    skeys = {tuple(row) for row in S}
    ov = len(rkeys & skeys)/max(1,len(skeys))  # lower is better
    return float(ov)

# ---------- load data ----------
real = pd.read_csv(real_path)
syn  = pd.read_csv(syn_path)

# ---------- compute Q ----------
cat_cols = real.select_dtypes(include=['object','category','bool']).columns.tolist()
num_cols = [c for c in real.columns if c not in cat_cols and pd.api.types.is_numeric_dtype(real[c])]

ks_vals = []
for c in num_cols:
    r = real[c].dropna().astype(float); s = syn[c].dropna().astype(float)
    if len(r)>30 and len(s)>30:
        ks_vals.append(ks_2samp(r, s).statistic)
Q_num = 1.0 - (np.mean(ks_vals) if ks_vals else 1.0)

tvd_vals = []
for c in cat_cols:
    tvd_vals.append(tvd(real[c].value_counts(normalize=True), syn[c].value_counts(normalize=True)))
Q_cat = 1.0 - (np.mean(tvd_vals) if tvd_vals else 0.0)

Q_range = np.mean([range_coverage(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_anom  = np.mean([anomaly_similarity(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 1.0
Q_corr  = corr_similarity(real, syn)
Q_div   = category_coverage(real, syn, cat_cols)

w = dict(num=0.20, cat=0.20, range=0.15, anom=0.15, corr=0.20, cov=0.10)
Q_overall = (w['num']*Q_num + w['cat']*Q_cat + w['range']*Q_range +
             w['anom']*Q_anom + w['corr']*Q_corr + w['cov']*Q_div)

# ---------- compute P ----------
P_DCR    = dcr_score(real, syn)
P_Qdelta = np.mean([quantile_diff_score(real[c].dropna().to_numpy(), syn[c].dropna().to_numpy()) for c in num_cols]) if num_cols else 0.5
P_noDup  = 1.0 - np.clip(duplicate_overlap(real, syn), 0, 1)
wp = dict(dcr=0.5, qd=0.3, dup=0.2)
P_overall = wp['dcr']*P_DCR + wp['qd']*P_Qdelta + wp['dup']*P_noDup

# ---------- report ----------
report = pd.DataFrame([{
    "dataset": Path(real_path).name,
    "synthetic": Path(syn_path).name,
    "Q_overall": round(Q_overall,3),
    "Q_num(1-KS)": round(Q_num,3),
    "Q_cat(1-TVD)": round(Q_cat,3),
    "Q_range": round(Q_range,3),
    "Q_anomaly": round(Q_anom,3),
    "Q_corr": round(Q_corr,3),
    "Q_diversity": round(Q_div,3),
    "P_overall": round(P_overall,3),
    "P_DCR": round(P_DCR,3),
    "P_Qdelta": round(P_Qdelta,3),
    "P_noDup": round(P_noDup,3),
    "n_real": len(real),
    "n_syn": len(syn),
    "num_cols_used": len(num_cols),
    "cat_cols_used": len(cat_cols)
}])

# Wide (original) – still saved to CSV
report.to_csv(out_csv, index=False)
print("\nSaved report (wide format) to:", out_csv)

# ---------- vertical tabular print ----------
vertical = report.T.reset_index()
vertical.columns = ["Metric", "Value"]
print("\n==== Metrics (vertical view) ====\n")
print(vertical.to_string(index=False))



Saved report (wide format) to: /content/drive/MyDrive/CTAB-GAN-Plus-main/CTAB-GAN-Plus-main/data/breast_cancer_CTABGAN_DP_QP_report.csv

==== Metrics (vertical view) ====

       Metric                                  Value
      dataset            breast-cancer-wisconsin.csv
    synthetic breast-cancer-wisconsin_CTABGAN_DP.csv
    Q_overall                                  0.814
  Q_num(1-KS)                                  0.754
 Q_cat(1-TVD)                                    1.0
      Q_range                                  0.953
    Q_anomaly                                  0.958
       Q_corr                                  0.382
  Q_diversity                                    1.0
    P_overall                                  0.866
        P_DCR                                    1.0
     P_Qdelta                                  0.555
      P_noDup                                    1.0
       n_real                                    683
        n_syn                   